In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
N_ranks = 4
orbital_period = 6000
folder = "results_1"

In [ ]:
losses = []
timestep = []
for rank in range(N_ranks):
    losses.append(np.loadtxt(folder + "/loss_rank"+str(rank)+".csv", delimiter=","))
    timestep.append(np.loadtxt(folder + "/time_at_loss_rank"+str(rank)+".csv", delimiter=","))

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(12, 6),dpi=150)
for time,loss in zip(timestep,losses):
    plt.plot((time - time[0]) / orbital_period,loss)
plt.xlabel("Completed Orbits")
plt.ylabel("Loss")
plt.yscale("log")
plt.legend(["Rank="+str(rank) for rank in range(N_ranks)])

In [ ]:
data_all_ranks = []
for rank in range(N_ranks):
    data_all_ranks.append(pd.read_csv(folder + "/" + str(rank)+".csv"))

In [ ]:
from create_plots import get_known_actor_comms_status
# Quantities we wanna track
quantities = ["temperature","state_of_charge","current_activity","known_actors"]

# Setup data frame to collect all data
df = pd.DataFrame(columns=("Time","ID"))
for idx,item in enumerate(quantities):
    names = []

    # Dataframe for this quantity
    small_df = pd.DataFrame(columns=("Time","ID"))
    
    plt.figure(figsize=(8, 2),dpi=150)

    # Get dat afrom all satellites and plot it
    for rank,data in enumerate(data_all_ranks):

        # Get time of each data point
        timesteps = data["timesteps"]

        # Get data
        if item == "known_actors":
            values = get_known_actor_comms_status(data[item])
        else:
            values = data[item]
        names.append(rank)

        # Collect data from this sat into big dataframe
        smaller_df = pd.DataFrame({"Time": timesteps,"ID": len(timesteps)*[rank],item: values})
        small_df = pd.concat([small_df,smaller_df])
        
        # Plot it :)
        plt.plot(timesteps,values)
        plt.xlabel("Time [s]")
        plt.ylabel(item.replace("_", " "))
        if item == "known_actors":
            plt.yticks([0, 1, 2, 3], ["None", "Svalbard", "Matera", "Maspalomas"])

    # Add a legend showing which satellite is which
    plt.legend(
            names,
            fontsize = 8,
            bbox_to_anchor=(0.5, 1.4),
            ncol=10,
            loc="upper center",
    )
    
    df = df.merge(small_df,how="right")

In [ ]:
from analysis_utils import get_analysis_df
N_timesteps = 128
df.Time = df.Time-df.Time.min() # convert to start t=0
df["Completed orbits"] = df.Time / orbital_period
analysis_df = get_analysis_df(df,df.Time.max() // N_timesteps,orbital_period)
analysis_df.head()

In [ ]:
sns.set(font_scale=1.8)
sns.set_theme(style="whitegrid")
melt_df = analysis_df[["Completed orbits",'# of Sats with commswindow']].melt('Completed orbits', var_name='Status', value_name='# of Satellites')
g = sns.catplot(x="Completed orbits", y="# of Satellites", hue='Status', data=melt_df, kind='bar',aspect=2)
g.set_xticklabels(rotation=90)

In [ ]:
sns.set(font_scale=1.8)
sns.set_theme(style="whitegrid")
melt_df = analysis_df[["Completed orbits",'Share Processing']].melt('Completed orbits', var_name='Status', value_name='Ratio of Satellites')
g = sns.catplot(x="Completed orbits", y="Ratio of Satellites", hue='Status', data=melt_df, kind='bar',aspect=2)
g.set_xticklabels(rotation=90)

In [ ]:
sns.set(rc={'figure.figsize':(12,6)})
sns.set(font_scale=1.8)
sns.set_theme(style="whitegrid")
sns.lineplot(data=df,x="Completed orbits",y="temperature",errorbar=("pi",100),estimator="median")
sns.lineplot(data=df,x="Completed orbits",y="temperature",errorbar=("pi",50))
sns.lineplot(data=df,x="Completed orbits",y="temperature",errorbar=("pi",25))
plt.legend(["Median","100% PI","_","50% PI","Mean","25% PI",]);
plt.ylabel("Temperature [K]")

In [ ]:
sns.lineplot(data=df,x="Completed orbits",y="state_of_charge",errorbar=("pi",100),estimator="median")
sns.lineplot(data=df,x="Completed orbits",y="state_of_charge",errorbar=("pi",50))
sns.lineplot(data=df,x="Completed orbits",y="state_of_charge",errorbar=("pi",25))
plt.legend(["Median","100% PI","_","50% PI","Mean","25% PI",]);
plt.ylabel("State of Charge")